In [6]:
pip install tensorflow numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


### GPU 확인

In [ ]:
pip install tensorflow-gpu


Defaulting to user installation because normal site-packages is not writeable
  ERROR: Command errored out with exit status 1:
   command: /Library/Developer/CommandLineTools/usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-install-bgzyodw0/tensorflow-gpu_323a8121775c42ceacbabd93db6300a4/setup.py'"'"'; __file__='"'"'/private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-install-bgzyodw0/tensorflow-gpu_323a8121775c42ceacbabd93db6300a4/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-wheel-y7hfvvoc
       cwd: /private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-install-bgzyodw0/tensor

In [4]:
pip install tqdm


Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 78 kB 8.0 MB/s eta 0:00:011
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install gym

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 721 kB 9.0 MB/s eta 0:00:01
  distutils: /private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-build-env-wp1puhl6/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-build-env-wp1puhl6/normal/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = False
  home = None
  root = None
  prefix = '/private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-build-env-wp1puhl6/normal'
  distutils: /private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-build-env-wp1puhl6/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  distutils: /private/var/folders/nd/hs1yql8s5k9bdqkwfv8sdngh0000gn/T/pip-build-env-wp1puhl6/overlay/lib/python3.9/site-packages
  sysconfig: /Library/Python/3.9/site-packages
  user = 

In [133]:
import os
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import gym
from gym import spaces

In [134]:
# GPU 사용 여부 확인
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))



Num GPUs Available:  0


## 2단계: 환경 정의
강화학습에서 사용할 배달 환경을 정의합니다.

In [154]:
class DeliveryEnv(gym.Env):
    def __init__(self, orders, riders, dist_mat):
        super(DeliveryEnv, self).__init__()
        self.orders = orders
        self.riders = riders
        self.dist_mat = dist_mat
        self.K = len(orders)

        # 상태 공간 정의: 주문의 상태 (0: 미처리, 1: 픽업 완료, 2: 배달 완료)
        self.observation_space = spaces.Box(low=0, high=2, shape=(self.K,), dtype=np.int32)

        # 행동 공간 정의: 각 주문의 픽업 또는 배달
        self.action_space = spaces.Discrete(2 * self.K)

        # 초기 상태 설정
        self.reset()

    def reset(self):
        self.state = np.zeros(self.K, dtype=np.int32)
        self.current_time = 0  # 현재 시간 초기화
        return self.state.astype(np.float32)

    def step(self, action):
        order_id = action % self.K
        is_pickup = action < self.K

        # 보상 및 종료 조건 초기화
        reward = 0
        done = False

        if is_pickup:
            if self.state[order_id] == 0 and self.current_time >= self.orders[order_id][1] + self.orders[order_id][6]:
                self.state[order_id] = 1
                reward += 1  # 픽업 성공 보상
        else:
            if self.state[order_id] == 1:
                delivery_time = self.current_time + self.get_travel_time(order_id, is_pickup=False)
                if delivery_time <= self.orders[order_id][8]:
                    self.state[order_id] = 2
                    reward += 10  # 배달 성공 보상
                self.current_time = delivery_time  # 현재 시간을 배달 시간으로 갱신

        # 모든 주문이 배달 완료되었는지 확인
        if np.all(self.state == 2):
            done = True

        return self.state.astype(np.float32), reward, done, {}

    def get_travel_time(self, order_id, is_pickup):
        if is_pickup:
            return 0  # 픽업은 즉시 완료
        else:
            # 배달 위치까지의 이동 시간 계산
            return int(np.round(self.dist_mat[order_id][order_id + self.K] / self.riders[0][1]))

    def render(self, mode='human'):
        pass

## 3단계: PPO 알고리즘 정의
PPO 알고리즘을 정의합니다.

In [155]:
class PPO:
    def __init__(self, env, gamma=0.99, lr=0.001, clip_ratio=0.2, update_steps=10):
        self.env = env
        self.gamma = gamma
        self.lr = lr
        self.clip_ratio = clip_ratio
        self.update_steps = update_steps

        # 배우 및 비평가 모델 정의
        self.actor_model = self.build_actor_model()
        self.critic_model = self.build_critic_model()

        self.actor_optimizer = Adam(learning_rate=self.lr)
        self.critic_optimizer = Adam(learning_rate=self.lr)

    def build_actor_model(self):
        model = Sequential([
            Input(shape=(self.env.observation_space.shape[0],)),
            Dense(64, activation='relu'),
            Dense(self.env.action_space.n, activation='softmax')
        ])
        return model

    def build_critic_model(self):
        model = Sequential([
            Input(shape=(self.env.observation_space.shape[0],)),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        return model

    def choose_action(self, state):
        state = np.expand_dims(state, axis=0)  # 상태를 (1, K) 형태로 확장
        state = tf.convert_to_tensor(state, dtype=tf.float32)  # 상태를 float32로 변환
        action_probs = self.actor_model(state)
        action = np.random.choice(self.env.action_space.n, p=action_probs.numpy().flatten())
        return action

    def learn(self, states, actions, rewards, next_states, dones):
        # Advantage 계산
        states = tf.convert_to_tensor(np.vstack(states), dtype=tf.float32)
        next_states = tf.convert_to_tensor(np.vstack(next_states), dtype=tf.float32)
        actions = tf.convert_to_tensor(np.array(actions), dtype=tf.int32)
        rewards = tf.convert_to_tensor(np.array(rewards), dtype=tf.float32)
        dones = tf.convert_to_tensor(np.array(dones), dtype=tf.float32)

        with tf.GradientTape() as tape:
            values = tf.squeeze(self.critic_model(states))
            next_values = tf.squeeze(self.critic_model(next_states))
            targets = rewards + self.gamma * next_values * (1 - dones)
            advantages = targets - values
            value_loss = tf.reduce_mean(tf.square(targets - values))

        grads = tape.gradient(value_loss, self.critic_model.trainable_variables)
        self.critic_optimizer.apply_gradients(zip(grads, self.critic_model.trainable_variables))

        # Actor 업데이트
        with tf.GradientTape() as tape:
            action_probs = self.actor_model(states)
            action_log_probs = tf.reduce_sum(tf.one_hot(actions, self.env.action_space.n) * tf.math.log(action_probs), axis=1)
            old_action_log_probs = tf.stop_gradient(action_log_probs)
            ratios = tf.exp(action_log_probs - old_action_log_probs)
            clipped_ratios = tf.clip_by_value(ratios, 1 - self.clip_ratio, 1 + self.clip_ratio)
            actor_loss = -tf.reduce_mean(tf.minimum(ratios * advantages, clipped_ratios * advantages))

        grads = tape.gradient(actor_loss, self.actor_model.trainable_variables)
        self.actor_optimizer.apply_gradients(zip(grads, self.actor_model.trainable_variables))

    def train(self, episodes=200, max_steps=200):
        for episode in range(episodes):
            state = self.env.reset()
            states, actions, rewards, next_states, dones = [], [], [], [], []

            for step in range(max_steps):
                action = self.choose_action(state)
                next_state, reward, done, _ = self.env.step(action)

                states.append(state)
                actions.append(action)
                rewards.append(reward)
                next_states.append(next_state)
                dones.append(done)

                state = next_state

                if done:
                    break

            self.learn(states, actions, rewards, next_states, dones)


## 데이터 로드

In [152]:
import os
import json

data_dir = 'test_data'
data_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.json')]

environments = []
for data_file in data_files:
    with open(data_file, 'r') as f:
        problem_data = json.load(f)
    K = problem_data['K']
    orders = problem_data['ORDERS']
    riders = problem_data['RIDERS']
    dist_mat = np.array(problem_data['DIST'])
    env = DeliveryEnv(orders, riders, dist_mat)
    environments.append(env)

print(f"Loaded {len(environments)} environments.")


Loaded 24 environments.


## 5단계: 모델 학습
각 환경에 대해 모델을 학습합니다.

In [184]:
import gym
from gym import spaces
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import os
import json

class DeliveryEnv(gym.Env):
    def __init__(self, orders, riders, dist_mat, max_state_size):
        super(DeliveryEnv, self).__init__()
        self.orders = orders
        self.riders = riders
        self.dist_mat = dist_mat
        self.K = len(orders)
        self.n_riders = len(riders)

        # 상태 공간 정의: 주문의 상태 (0: 미처리, 1: 픽업 완료, 2: 배달 완료)
        self.observation_space = spaces.Box(low=0, high=2, shape=(max_state_size,), dtype=np.float32)

        # 행동 공간 정의: 각 주문의 픽업 또는 배달
        self.action_space = spaces.Discrete(max_state_size)

        # 초기 상태 설정
        self.reset()

    def reset(self):
        self.state = np.zeros(self.observation_space.shape[0], dtype=np.float32)
        self.current_time = 0  # 현재 시간 초기화
        return self.state

    def step(self, action):
        order_id = action % self.K
        is_pickup = action < self.K

        # 보상 및 종료 조건 초기화
        reward = 0
        done = False

        if is_pickup:
            if self.state[order_id] == 0 and self.current_time >= self.orders[order_id][1] + self.orders[order_id][6]:
                self.state[order_id] = 1
                reward += 1  # 픽업 성공 보상
        else:
            if self.state[order_id] == 1:
                travel_time = self.get_travel_time(order_id, is_pickup=False)
                delivery_time = self.current_time + travel_time
                if delivery_time <= self.orders[order_id][8]:
                    self.state[order_id] = 2
                    reward += 10  # 배달 성공 보상
                self.current_time = delivery_time  # 현재 시간을 배달 시간으로 갱신

        # 모든 주문이 배달 완료되었는지 확인
        if np.all(self.state[:self.K] == 2):
            done = True

        return self.state, reward, done, {}

    def get_travel_time(self, order_id, is_pickup):
        if is_pickup:
            return 0  # 픽업은 즉시 완료
        else:
            # 배달 위치까지의 이동 시간 계산
            return int(np.round(self.dist_mat[order_id][order_id + self.K] / self.riders[0][1]))

    def render(self, mode='human'):
        pass


class PPO:
    def __init__(self, env, state_size, gamma=0.99, lr=0.001, clip_ratio=0.2, update_steps=10):
        self.env = env
        self.gamma = gamma
        self.lr = lr
        self.clip_ratio = clip_ratio
        self.update_steps = update_steps

        # 배우 및 비평가 모델 정의
        self.actor_model = self.build_actor_model(state_size)
        self.critic_model = self.build_critic_model(state_size)

        self.actor_optimizer = Adam(learning_rate=self.lr)
        self.critic_optimizer = Adam(learning_rate=self.lr)

    def build_actor_model(self, state_size):
        model = Sequential([
            Input(shape=(state_size,)),
            Dense(64, activation='relu'),
            Dense(self.env.action_space.n, activation='softmax')
        ])
        return model

    def build_critic_model(self, state_size):
        model = Sequential([
            Input(shape=(state_size,)),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        return model

    def choose_action(self, state):
        state = np.expand_dims(state, axis=0)  # 상태를 (1, K) 형태로 확장
        state = tf.convert_to_tensor(state, dtype=tf.float32)  # 상태를 float32로 변환
        action_probs = self.actor_model(state, training=False).numpy().flatten()
        assert len(action_probs) == self.env.action_space.n, "action_probs와 action_space.n의 크기가 일치하지 않습니다."
        action = np.random.choice(self.env.action_space.n, p=action_probs)
        return action

    def learn(self, states, actions, rewards, next_states, dones):
        # Advantage 계산
        states = tf.convert_to_tensor(np.vstack(states), dtype=tf.float32)
        next_states = tf.convert_to_tensor(np.vstack(next_states), dtype=tf.float32)
        actions = tf.convert_to_tensor(np.array(actions), dtype=tf.int32)
        rewards = tf.convert_to_tensor(np.array(rewards), dtype=tf.float32)
        dones = tf.convert_to_tensor(np.array(dones), dtype=tf.float32)

        with tf.GradientTape() as tape:
            values = tf.squeeze(self.critic_model(states, training=True))
            next_values = tf.squeeze(self.critic_model(next_states, training=True))
            targets = rewards + self.gamma * next_values * (1 - dones)
            advantages = targets - values
            value_loss = tf.reduce_mean(tf.square(targets - values))

        grads = tape.gradient(value_loss, self.critic_model.trainable_variables)
        self.critic_optimizer.apply_gradients(zip(grads, self.critic_model.trainable_variables))

        # Actor 업데이트
        with tf.GradientTape() as tape:
            action_probs = self.actor_model(states, training=True)
            action_log_probs = tf.reduce_sum(tf.one_hot(actions, self.env.action_space.n) * tf.math.log(action_probs), axis=1)
            old_action_log_probs = tf.stop_gradient(action_log_probs)
            ratios = tf.exp(action_log_probs - old_action_log_probs)
            clipped_ratios = tf.clip_by_value(ratios, 1 - self.clip_ratio, 1 + self.clip_ratio)
            actor_loss = -tf.reduce_mean(tf.minimum(ratios * advantages, clipped_ratios * advantages))

        grads = tape.gradient(actor_loss, self.actor_model.trainable_variables)
        self.actor_optimizer.apply_gradients(zip(grads, self.actor_model.trainable_variables))

    def train(self, episodes=1000, max_steps=1000):
        for episode in range(episodes):
            state = self.env.reset()
            states, actions, rewards, next_states, dones = [], [], [], [], []

            for step in range(max_steps):
                action = self.choose_action(state)
                next_state, reward, done, _ = self.env.step(action)

                states.append(state)
                actions.append(action)
                rewards.append(reward)
                next_states.append(next_state)
                dones.append(done)

                state = next_state

                if done:
                    break

            self.learn(states, actions, rewards, next_states, dones)

def train_on_multiple_environments(agent, envs, data_files, episodes_per_env=200):
    for env in tqdm(envs, desc="Training on Environments"):
        agent.env = env
        agent.actor_model = agent.build_actor_model(agent.env.observation_space.shape[0])
        agent.critic_model = agent.build_critic_model(agent.env.observation_space.shape[0])
        agent.actor_optimizer = Adam(learning_rate=agent.lr)
        agent.critic_optimizer = Adam(learning_rate=agent.lr)
        agent.train(episodes=episodes_per_env)

    agent.actor_model.save("actor_delivery_combined.keras")
    agent.critic_model.save("critic_delivery_combined.keras")

# 예제 실행
data_dir = 'test_data'  # 데이터셋이 있는 디렉토리
data_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.json')]

environments = []
max_state_size = 600

for data_file in data_files:
    with open(data_file, 'r') as f:
        problem_data = json.load(f)
    K = problem_data['K']
    max_state_size = max(max_state_size, 2 * K)
    orders = problem_data['ORDERS']
    riders = problem_data['RIDERS']
    dist_mat = np.array(problem_data['DIST'])
    env = DeliveryEnv(orders, riders, dist_mat, max_state_size)
    environments.append(env)

ppo_agent = PPO(environments[0], max_state_size)  # 초기 환경을 하나 선택하여 에이전트 생성
train_on_multiple_environments(ppo_agent, environments, data_files)


Training on Environments: 100%|██████████| 24/24 [1:26:14<00:00, 215.59s/it]


## 모델 확인 작업

In [176]:
import os
import json
import numpy as np
import tensorflow as tf
from keras.models import load_model

# 학습된 모델 로드
actor_model = load_model('actor_delivery_combined.keras')
critic_model = load_model('critic_delivery_combined.keras')


### 환경 클래스 정의

In [177]:
class DeliveryEnv:
    def __init__(self, orders, riders, dist_mat):
        self.orders = orders
        self.riders = riders
        self.dist_mat = dist_mat
        self.reset()

    def reset(self):
        self.current_position = [order[2:4] for order in self.orders]  # 시작 위치를 주문의 픽업 위치로 설정
        self.current_time = [0 for _ in self.orders]  # 각 주문의 시작 시간을 0으로 설정
        self.done = False
        return self.current_state()

    def current_state(self):
        # 현재 상태를 반환하는 함수
        state = np.concatenate([np.array(self.current_position).flatten(), np.array(self.current_time).flatten()])
        return state

    def step(self, action):
        # 액션을 수행하여 다음 상태, 보상, 완료 여부를 반환하는 함수
        rider_idx, next_pos = divmod(action, len(self.orders))
        dist = self.dist_mat[self.current_position[rider_idx], next_pos]
        self.current_position[rider_idx] = self.orders[next_pos][4:6]
        self.current_time[rider_idx] += dist / self.riders[rider_idx][1]
        reward = -dist  # 거리만큼 보상을 마이너스로 설정 (거리가 짧을수록 보상이 큼)
        done = all([time > order[7] for time, order in zip(self.current_time, self.orders)])
        return self.current_state(), reward, done


### 모델 평가 함수 정의

In [178]:
def evaluate_model(env, actor_model, critic_model):
    state = env.reset()
    total_reward = 0
    done = False

    while not done:
        state = np.reshape(state, [1, len(state)])
        action_probs = actor_model.predict(state, verbose=0)
        action = np.argmax(action_probs[0])
        next_state, reward, done = env.step(action)
        total_reward += reward
        state = next_state

    return total_reward


### 여러 데이터셋 로드

In [179]:
data_dir = 'test_data'
data_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.json')]

environments = []
for data_file in data_files:
    with open(data_file, 'r') as f:
        problem_data = json.load(f)
    K = problem_data['K']
    orders = problem_data['ORDERS']
    riders = problem_data['RIDERS']
    dist_mat = np.array(problem_data['DIST'])
    env = DeliveryEnv(orders, riders, dist_mat)
    environments.append(env)

print(f"Loaded {len(environments)} environments.")


Loaded 24 environments.


### 모든 환경에서 모델 평가

In [185]:
total_rewards = []
for env in environments:
    total_reward = evaluate_model(env, actor_model, critic_model)
    total_rewards.append(total_reward)


KeyboardInterrupt: 

결과 정리 및 출력

In [ ]:
average_reward = np.mean(total_rewards)
print(f"평균 총 보상: {average_reward}")
print(f"각 환경별 총 보상: {total_rewards}")


In [183]:
import gym
from gym import spaces
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tqdm import tqdm
import os
import json

class DeliveryEnv(gym.Env):
    def __init__(self, orders, riders, dist_mat, max_state_size):
        super(DeliveryEnv, self).__init__()
        self.orders = orders
        self.riders = riders
        self.dist_mat = dist_mat
        self.K = len(orders)
        self.n_riders = len(riders)
        self.max_state_size = max_state_size

        # 상태 공간 정의: 주문의 상태 (0: 미처리, 1: 픽업 완료, 2: 배달 완료)
        self.observation_space = spaces.Box(low=0, high=2, shape=(max_state_size,), dtype=np.float32)

        # 행동 공간 정의: 각 주문의 픽업 또는 배달
        self.action_space = spaces.Discrete(max_state_size)

        # 초기 상태 설정
        self.reset()

    def reset(self):
        self.state = np.zeros(self.observation_space.shape[0], dtype=np.float32)
        self.current_time = 0  # 현재 시간 초기화
        return self.state

    def step(self, action):
        order_id = action % self.K
        is_pickup = action < self.K

        # 보상 및 종료 조건 초기화
        reward = 0
        done = False

        if is_pickup:
            if self.state[order_id] == 0 and self.current_time >= self.orders[order_id][1] + self.orders[order_id][6]:
                self.state[order_id] = 1
                reward += 1  # 픽업 성공 보상
        else:
            if self.state[order_id] == 1:
                travel_time = self.get_travel_time(order_id, is_pickup=False)
                delivery_time = self.current_time + travel_time
                if delivery_time <= self.orders[order_id][8]:
                    self.state[order_id] = 2
                    reward += 10  # 배달 성공 보상
                self.current_time = delivery_time  # 현재 시간을 배달 시간으로 갱신

        # 모든 주문이 배달 완료되었는지 확인
        if np.all(self.state[:self.K] == 2):
            done = True

        return self.state, reward, done, {}

    def get_travel_time(self, order_id, is_pickup):
        if is_pickup:
            return 0  # 픽업은 즉시 완료
        else:
            # 배달 위치까지의 이동 시간 계산
            return int(np.round(self.dist_mat[order_id][order_id + self.K] / self.riders[0][1]))

    def render(self, mode='human'):
        pass

class PPO:
    def __init__(self, env, state_size, gamma=0.99, lr=0.001, clip_ratio=0.2, update_steps=10):
        self.env = env
        self.gamma = gamma
        self.lr = lr
        self.clip_ratio = clip_ratio
        self.update_steps = update_steps

        # 배우 및 비평가 모델 정의
        self.actor_model = self.build_actor_model(state_size)
        self.critic_model = self.build_critic_model(state_size)

        self.actor_optimizer = Adam(learning_rate=self.lr)
        self.critic_optimizer = Adam(learning_rate=self.lr)

    def build_actor_model(self, state_size):
        model = Sequential([
            Input(shape=(state_size,)),
            Dense(64, activation='relu'),
            Dense(self.env.action_space.n, activation='softmax')
        ])
        return model

    def build_critic_model(self, state_size):
        model = Sequential([
            Input(shape=(state_size,)),
            Dense(64, activation='relu'),
            Dense(1)
        ])
        return model

    def choose_action(self, state):
        state = np.expand_dims(state, axis=0)  # 상태를 (1, K) 형태로 확장
        state = tf.convert_to_tensor(state, dtype=tf.float32)  # 상태를 float32로 변환
        action_probs = self.actor_model(state, training=False).numpy().flatten()
        assert len(action_probs) == self.env.action_space.n, "action_probs와 action_space.n의 크기가 일치하지 않습니다."
        action = np.random.choice(self.env.action_space.n, p=action_probs)
        return action

    def learn(self, states, actions, rewards, next_states, dones):
        # Advantage 계산
        states = tf.convert_to_tensor(np.vstack(states), dtype=tf.float32)
        next_states = tf.convert_to_tensor(np.vstack(next_states), dtype=tf.float32)
        actions = tf.convert_to_tensor(np.array(actions), dtype=tf.int32)
        rewards = tf.convert_to_tensor(np.array(rewards), dtype=tf.float32)
        dones = tf.convert_to_tensor(np.array(dones), dtype=tf.float32)

        with tf.GradientTape() as tape:
            values = tf.squeeze(self.critic_model(states, training=True))
            next_values = tf.squeeze(self.critic_model(next_states, training=True))
            targets = rewards + self.gamma * next_values * (1 - dones)
            advantages = targets - values
            value_loss = tf.reduce_mean(tf.square(targets - values))

        grads = tape.gradient(value_loss, self.critic_model.trainable_variables)
        self.critic_optimizer.apply_gradients(zip(grads, self.critic_model.trainable_variables))

        # Actor 업데이트
        with tf.GradientTape() as tape:
            action_probs = self.actor_model(states, training=True)
            action_log_probs = tf.reduce_sum(tf.one_hot(actions, self.env.action_space.n) * tf.math.log(action_probs), axis=1)
            old_action_log_probs = tf.stop_gradient(action_log_probs)
            ratios = tf.exp(action_log_probs - old_action_log_probs)
            clipped_ratios = tf.clip_by_value(ratios, 1 - self.clip_ratio, 1 + self.clip_ratio)
            actor_loss = -tf.reduce_mean(tf.minimum(ratios * advantages, clipped_ratios * advantages))

        grads = tape.gradient(actor_loss, self.actor_model.trainable_variables)
        self.actor_optimizer.apply_gradients(zip(grads, self.actor_model.trainable_variables))

    def train(self, episodes=1000, max_steps=1000):
        for episode in range(episodes):
            state = self.env.reset()
            states, actions, rewards, next_states, dones = [], [], [], [], []

            for step in range(max_steps):
                action = self.choose_action(state)
                next_state, reward, done, _ = self.env.step(action)

                states.append(state)
                actions.append(action)
                rewards.append(reward)
                next_states.append(next_state)
                dones.append(done)

                state = next_state

                if done:
                    break

            self.learn(states, actions, rewards, next_states, dones)

def train_on_multiple_environments(agent, envs, episodes_per_env=200):
    for env in tqdm(envs, desc="Training on Environments"):
        agent.env = env
        agent.actor_model = agent.build_actor_model(agent.env.observation_space.shape[0])
        agent.critic_model = agent.build_critic_model(agent.env.observation_space.shape[0])
        agent.actor_optimizer = Adam(learning_rate=agent.lr)
        agent.critic_optimizer = Adam(learning_rate=agent.lr)
        agent.train(episodes=episodes_per_env)

    agent.actor_model.save("actor_delivery_combined.h5")
    agent.critic_model.save("critic_delivery_combined.h5")

# 예제 실행
data_dir = 'test_data'  # 데이터셋이 있는 디렉토리
data_files = [os.path.join(data_dir, file) for file in os.listdir(data_dir) if file.endswith('.json')]

environments = []
max_state_size = 0

for data_file in data_files:
    with open(data_file, 'r') as f:
        problem_data = json.load(f)
    K = problem_data['K']
    max_state_size = max(max_state_size, 2 * K)
    orders = problem_data['ORDERS']
    riders = problem_data['RIDERS']
    dist_mat = np.array(problem_data['DIST'])
    env = DeliveryEnv(orders, riders, dist_mat, max_state_size)
    environments.append(env)

ppo_agent = PPO(environments[0], max_state_size)  # 초기 환경을 하나 선택하여 에이전트 생성
train_on_multiple_environments(ppo_agent, environments)

# 모델 평가
actor_model = load_model("actor_delivery_combined.h5")
critic_model = load_model("critic_delivery_combined.h5")

def evaluate_model(env, actor_model, critic_model):
    state = env.reset()
    done = False
    total_reward = 0
    while not done:
        state = np.reshape(state, [1, env.observation_space.shape[0]])
        action_probs = actor_model.predict(state, verbose=0)
        action = np.argmax(action_probs[0])
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = next_state
    return total_reward

# 모든 환경에서 평가
total_rewards = []
for env in environments:
    total_reward = evaluate_model(env, actor_model, critic_model)
    total_rewards.append(total_reward)

print("Total rewards from each environment:", total_rewards)


Training on Environments: 100%|██████████| 24/24 [38:58:32<00:00, 5846.34s/it]    


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_628" is incompatible with the layer: expected axis -1 of input shape to have value 600, but received input with shape (1, 200)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 200), dtype=float32)
  • training=False
  • mask=None